In [ ]:
# Model 8 - CNN- LSTM Model
# Define the RNNCNN1 class that inherits from ModelBuilder
class RNNCNN1(ModelBuilder):

    # Define the model structure with parameters for LSTM cells, dense layer neurons, and dropout rate
    def define_model(self, lstm_cells=64, dense_neurons=64, dropout=0.25):

        # Initialize a sequential model
        model = Sequential()

        # Add the first TimeDistributed Conv2D layer with 16 filters, kernel size of 3x3, 'same' padding, and ReLU activation
        model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'),
                                  input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(TimeDistributed(BatchNormalization()))  # Add TimeDistributed BatchNormalization
        model.add(TimeDistributed(MaxPooling2D((2, 2))))  # Add TimeDistributed MaxPooling2D with pool size of 2x2
        
        # Add the second TimeDistributed Conv2D layer with 32 filters and kernel size of 3x3
        model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))  # Add TimeDistributed BatchNormalization
        model.add(TimeDistributed(MaxPooling2D((2, 2))))  # Add TimeDistributed MaxPooling2D with pool size of 2x2
        
        # Add the third TimeDistributed Conv2D layer with 64 filters and kernel size of 3x3
        model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))  # Add TimeDistributed BatchNormalization
        model.add(TimeDistributed(MaxPooling2D((2, 2))))  # Add TimeDistributed MaxPooling2D with pool size of 2x2
        
        # Add the fourth TimeDistributed Conv2D layer with 128 filters and kernel size of 3x3
        model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))  # Add TimeDistributed BatchNormalization
        model.add(TimeDistributed(MaxPooling2D((2, 2))))  # Add TimeDistributed MaxPooling2D with pool size of 2x2
        
        # Add the fifth TimeDistributed Conv2D layer with 256 filters and kernel size of 3x3
        model.add(TimeDistributed(Conv2D(256, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))  # Add TimeDistributed BatchNormalization
        model.add(TimeDistributed(MaxPooling2D((2, 2))))  # Add TimeDistributed MaxPooling2D with pool size of 2x2
        
        # Flatten the output from the convolutional layers to feed into the LSTM layer
        model.add(TimeDistributed(Flatten()))

        # Add LSTM layer with the specified number of LSTM cells
        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))  # Add dropout for regularization
        
        # Add a dense layer with the specified number of neurons and ReLU activation
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(Dropout(dropout))  # Add dropout for regularization
        
        # Add the output layer with the number of classes and softmax activation for classification
        model.add(Dense(self.num_classes, activation='softmax'))
        
        # Define the optimizer and compile the model with categorical crossentropy loss and accuracy metric
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        # Return the compiled model
        return model

This code defines a hybrid Convolutional Neural Network (CNN) and Long Short-Term Memory (LSTM) network within the RNNCNN1 class, inheriting from ModelBuilder. Here's a breakdown of the code:

**Key Features:**
- **1.TimeDistributed Layers:**
- Wraps CNN layers to process sequential data (video frames in this context).
- ** 2.CNN Component:**
- Multiple Conv2D layers with increasing filter counts (16, 32, 64, 128, 256) for hierarchical feature extraction.
- Batch Normalization (BatchNormalization) after each Conv2D layer for faster convergence.
- Max Pooling (MaxPooling2D) for spatial dimensionality reduction.
- **3.Flattening and Transition to RNN:**
- After the CNN feature extraction, outputs are flattened using TimeDistributed(Flatten()).
- **4.LSTM Layer:**
- Processes the sequential features extracted from the CNN layers.

- Number of LSTM cells is configurable through lstm_cells.
- **5.Dense Layers and Dropout:**
- Dense layer for classification after LSTM with a configurable number of neurons (dense_neurons).
- Dropout is applied after LSTM and dense layers for regularization.
- **6.Output Layer:**
- Final dense layer with softmax activation for multi-class classification (num_classes classes).
- **7.Compilation:**
- Adam optimizer.
- categorical_crossentropy loss and categorical_accuracy metrics.

 
















 






In [ ]:
rnn_cnn1=RNNCNN1()
rnn_cnn1.initialize_path(project_folder)
rnn_cnn1.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn1.initialize_hyperparams(frames_to_sample=18,batch_size=20,num_epochs=20)
rnn_cnn1_model=rnn_cnn1.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn1_model.summary()

- **the code initializes, configures, and defines an RNN-CNN model for processing sequential image data, then prints its architecture summary..**



In [ ]:
print("Total Params:", rnn_cnn1_model.count_params())
history_model8=rnn_cnn1.train_model(rnn_cnn1_model,augment_data=True)

In [ ]:
plot(history_model8)


In [ ]:
# Define the ModelBuilderMoreAugmentation class with ABCMeta as metaclass
class ModelBuilderMoreAugmentation(metaclass=abc.ABCMeta):
    
    # Initialize paths for training and validation data
    def initialize_path(self, project_folder):
        # Shuffle and read training and validation file lists
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path = project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
    # Initialize image properties like height, width, channels, classes, and total frames
    def initialize_image_properties(self, image_height=100, image_width=100):
        self.image_height = image_height
        self.image_width = image_width
        self.channels = 3
        self.num_classes = 5
        self.total_frames = 30
          
    # Initialize hyperparameters like frames to sample, batch size, and number of epochs
    def initialize_hyperparams(self, frames_to_sample=30, batch_size=20, num_epochs=20):
        self.frames_to_sample = frames_to_sample
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
    # Data generator function to yield batches of data and labels
    def generator(self, source_path, folder_list, augment=False):
        img_idx = np.round(np.linspace(0, self.total_frames-1, self.frames_to_sample)).astype(int)
        batch_size = self.batch_size
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(t) // batch_size
        
            for batch in range(num_batches): 
                batch_data, batch_labels = self.one_batch_data(source_path, t, batch, batch_size, img_idx, augment)
                yield batch_data, batch_labels 

            remaining_seq = len(t) % batch_size
        
            if remaining_seq != 0:
                batch_data, batch_labels = self.one_batch_data(source_path, t, num_batches, batch_size, img_idx, augment, remaining_seq)
                yield batch_data, batch_labels 
    
    # Prepare one batch of data with optional augmentation
    def one_batch_data(self, source_path, t, batch, batch_size, img_idx, augment, remaining_seq=0):
    
        seq_len = remaining_seq if remaining_seq else batch_size
    
        batch_data = np.zeros((seq_len, len(img_idx), self.image_height, self.image_width, self.channels)) 
        batch_labels = np.zeros((seq_len, self.num_classes)) 
    
        if augment:
            batch_data_aug = np.zeros((seq_len, len(img_idx), self.image_height, self.image_width, self.channels))

        for folder in range(seq_len): 
            imgs = os.listdir(source_path + '/' + t[folder + (batch * batch_size)].split(';')[0]) 
            for idx, item in enumerate(img_idx): 
                image = imageio.imread(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)
                image_resized = resize(image, (self.image_height, self.image_width, 3))

                batch_data[folder, idx, :, :, 0] = image_resized[:, :, 0] / 255
                batch_data[folder, idx, :, :, 1] = image_resized[:, :, 1] / 255
                batch_data[folder, idx, :, :, 2] = image_resized[:, :, 2] / 255
            
                if augment:
                    shifted = cv2.warpAffine(image, 
                                             np.float32([[1, 0, np.random.randint(-30, 30)], [0, 1, np.random.randint(-30, 30)]]), 
                                             (image.shape[1], image.shape[0]))
                    
                    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)

                    x0, y0 = np.argwhere(gray > 0).min(axis=0)
                    x1, y1 = np.argwhere(gray > 0).max(axis=0) 
                    
                    cropped = shifted[x0:x1, y0:y1, :]
                    
                    image_resized = resize(cropped, (self.image_height, self.image_width, 3))
                    
                    M = cv2.getRotationMatrix2D((self.image_width // 2, self.image_height // 2),
                                                np.random.randint(-10, 10), 1.0)
                    rotated = cv2.warpAffine(image_resized, M, (self.image_width, self.image_height))
                    
                    batch_data_aug[folder, idx, :, :, 0] = rotated[:, :, 0] / 255
                    batch_data_aug[folder, idx, :, :, 1] = rotated[:, :, 1] / 255
                    batch_data_aug[folder, idx, :, :, 2] = rotated[:, :, 2] / 255
            
            batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1
    
        if augment:
            batch_data = np.concatenate([batch_data, batch_data_aug])
            batch_labels = np.concatenate([batch_labels, batch_labels])

        return batch_data, batch_labels
    
    # Train the model using the generator, with optional data augmentation
    def train_model(self, model, augment_data=False):
        train_generator = self.generator(self.train_path, self.train_doc, augment=augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)

        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ', '').replace(':', '_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)
        
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
        callbacks_list = [checkpoint, LR]

        if (self.num_train_sequences % self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences / self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences // self.batch_size) + 1

        if (self.num_val_sequences % self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences / self.batch_size)
        else:
            validation_steps = (self.num_val_sequences // self.batch_size) + 1
    
        history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                                      callbacks=callbacks_list, validation_data=val_generator, 
                                      validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        return history

    # Abstract method to define the model, to be implemented by subclasses
    @abc.abstractmethod
    def define_model(self):
        pass

- **Objective:** The ModelBuilderMoreAugmentation class provides a framework to build and train deep learning models for image-based tasks, with built-in support for data augmentation and generators to handle large datasets efficiently.

**Key Components:**

 **1.Initialization Functions:**

- initialize_path(project_folder): Reads training and validation datasets (train.csv and val.csv), sets file paths, and calculates the number of sequences.
- initialize_image_properties(image_height, image_width): Sets image resolution, channels, classes, and total frames per sequence.
- initialize_hyperparams(frames_to_sample, batch_size, num_epochs): Configures hyperparameters such as the number of frames, batch size, and training epochs.
  
 **2.Data Generator (generator):**

- Dynamically loads and processes batches of data to avoid memory overload.
- Performs optional augmentation (e.g., translation, cropping, rotation).
- Yields batches of data and labels to the training loop.
  
 **3. Augmentation Logic (one_batch_data):**

- Applies random transformations like shifting, cropping, and rotation to augment training data.
- Ensures each augmented batch has corresponding labels.
  
 **4.Model Training (train_model):**

- Trains the model using the Keras fit_generator method with training and validation generators.
- Includes callbacks such as ModelCheckpoint for saving models and ReduceLROnPlateau for dynamic learning rate adjustments.
  
 **5.Abstract Method (define_model):**

- Provides a blueprint for subclasses to implement specific model architectures.
 

In [ ]:
# Create an instance of the test generator
test_generator = Test()

# Initialize paths, image properties, and hyperparameters
test_generator.initialize_path(project_folder)
test_generator.initialize_image_properties(image_height=160, image_width=160)
test_generator.initialize_hyperparams(frames_to_sample=30, batch_size=3, num_epochs=1)

# Generate a batch of data with augmentation
g = test_generator.generator(test_generator.val_path, test_generator.val_doc, augment=True)
batch_data, batch_labels = next(g)

# Visualize the augmented data
fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].imshow(batch_data[0, 29, :, :, :])
axes[1].imshow(batch_data[3, 29, :, :, :])
plt.show()

-- **Objective:**
- Evaluate the data generators ability to preprocess and augment image data for a sequence-based deep learning task.
  **Steps:**

**1.Setup:**

- Created an instance of the Test generator class.
- Initialized paths, image properties, and hyperparameters for the test dataset.
  
**2.Data Generation:**

- Used the generator to fetch a batch of augmented validation data.
- Retrieved batch_data (augmented images) and batch_labels (corresponding labels).

**3.Visualization:**

- Displayed two augmented frames from the batch to confirm transformations (e.g., cropping, rotation).




In [ ]:
 Model 9 is comparable to Model 2 with its 160x160 image resolution and augmentation (3,3,3) filter.

# Define the ModelConv3D9 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D9(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for filter size, dense neurons, and dropout rate
    def define_model(self, filter_size=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First Conv3D layer with 16 filters
        model.add(Conv3D(16, filter_size, padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Second Conv3D layer with 32 filters
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Third Conv3D layer with 64 filters
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Fourth Conv3D layer with 128 filters
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

**Expected Changes
 **1.Higher Resolution (160x160):**

- **Impact:** Captures more detailed spatial information, potentially improving accuracy, especially for datasets where fine-grained features are important.
- **Trade-off:** Increased computational cost and memory usage.
  
 **Smaller Filter Size (3x3x3):**

- **Impact:** Focuses on local features with smaller receptive fields, improving feature extraction for detailed patterns.
- **Trade-off:** May require more layers to capture global patterns effectively.
  
 **Additional Conv3D Layers:**

- **Impact:** Allows the model to learn more complex feature hierarchies by stacking layers.
- **Trade-off:** Increased training time and potential risk of overfitting if the dataset is small.
  
 **Dense Layers and Dropout:**

- **Impact:** Adding two dense layers with a smaller dropout rate (0.25) can enhance feature learning and reduce underfitting.
- **Trade-off:** May slightly increase overfitting risk if dropout is too low.
  
 **Augmentation (Model 9):**

- **Impact:** Augmentation in Model 9 introduces robustness to variations (e.g., translation, rotation), likely improving generalization.
- **Trade-off:** Requires additional computation during data preprocessing.



**Applications and Use Cases**
- **Model 8:** Suitable for scenarios with limited computational resources or datasets with lower resolution or less variation.
- **Model 9:** Designed for larger, more complex datasets where higher resolution and robust augmentation can enhance performance.

**Conclusion**
ModelConv3D9 is an improved and more computationally intensive version of ModelConv3D8, aiming for better performance through higher resolution, smaller filters, additional layers, and aggressive augmentation. It is ideal for scenarios requiring higher accuracy and robust feature learning. However, it demands more resources and careful regularization to prevent overfitting.

In [ ]:
conv_3d9=ModelConv3D9()
conv_3d9.initialize_path(project_folder)
conv_3d9.initialize_image_properties(image_height=160,image_width=160)
conv_3d9.initialize_hyperparams(frames_to_sample=20,batch_size=20,num_epochs=20)
conv_3d9_model=conv_3d9.define_model(dense_neurons=256,dropout=0.5)
conv_3d9_model.summary()

In [ ]:
print("Total Params:", conv_3d9_model.count_params())
history_model9=conv_3d9.train_model(conv_3d9_model,augment_data=True)

In [ ]:
# Similar to the Model 3, the Model 10 has an augmentation (2,2,2) filter and a 120x120 image resolution.

# Define the ModelConv3D10 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D10(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for filter size, dense neurons, and dropout rate
    def define_model(self, filter_size=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First Conv3D layer with 16 filters
        model.add(Conv3D(16, filter_size, padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Second Conv3D layer with 32 filters
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Third Conv3D layer with 64 filters
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Fourth Conv3D layer with 128 filters
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

**Expected Changes and Impact**
 **1.Image Resolution (160x160 vs. 120x120):**

- **Model 9 Impact:** Captures more spatial details, leading to better performance for datasets where fine-grained features are essential.
- **Model 10 Impact:** Lower resolution reduces computational overhead, making it more suitable for faster training and datasets where high resolution isn't critical.
  
 **2.Filter Size (3,3,3 vs. 2,2,2):**

- **Model 9 Impact:** Larger filters cover a wider receptive field, allowing the model to capture more global patterns in fewer layers.
- **Model 10 Impact:** Smaller filters focus on local features and require deeper architectures to achieve the same level of abstraction.
  
 **3.Augmentation (Assumed Difference):**

- **Model 9:** More aggressive augmentation introduces robustness to noise and variations, improving generalization.
- **Model 10:** May rely on less augmentation, which can lead to faster training but slightly reduced robustness to data variations.
  
 **4.Resource Requirements:**

- **Model 9:** Requires more GPU memory and processing time due to higher resolution and more complex filters.
- **Model 10:** More lightweight, suitable for scenarios with limited computational resources.

| **Scenario**                     | **ModelConv3D9**                    | **ModelConv3D10**                    |
|----------------------------------|-------------------------------------|-------------------------------------|
| **High-resolution video input**   | Recommended for extracting finer details. | May miss finer spatial details.    |
| **Low-resolution datasets**       | Overkill for such cases.             | Ideal for low-resolution datasets.  |
| **Limited computational resources** | May cause resource bottlenecks.      | More efficient for constrained resources. |
| **Dataset with large variations**  | Augmentation helps generalization.   | Less robust to variations.          |


**Conclusion**
- **ModelConv3D9:** Best suited for high-resolution datasets or cases requiring fine-grained feature extraction but at the cost of increased computational resources.
- **ModelConv3D10:** A lightweight alternative for datasets with lower resolutions or computational constraints, but may sacrifice performance in highly detailed

In [ ]:
conv_3d10=ModelConv3D10()
conv_3d10.initialize_path(project_folder)
conv_3d10.initialize_image_properties(image_height=120,image_width=120)
conv_3d10.initialize_hyperparams(frames_to_sample=16,batch_size=30,num_epochs=25)
conv_3d10_model=conv_3d10.define_model(filtersize=(2,2,2),dense_neurons=256,dropout=0.5)
conv_3d10_model.summary()

In [ ]:
print("Total Params:", conv_3d10_model.count_params())
history_model10=conv_3d10.train_model(conv_3d10_model,augment_data=True)

In [ ]:
plot(history_model10)


In [ ]:
# Model 11 with Augmentation
# Adding more layers - Similar to model 4

# Define the ModelConv3D11 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D11(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for filter size, dense neurons, and dropout rate
    def define_model(self, filter_size=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First block: two Conv3D layers with 16 filters
        model.add(Conv3D(16, filter_size, padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(16, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Second block: two Conv3D layers with 32 filters
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Third block: two Conv3D layers with 64 filters
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Fourth block: two Conv3D layers with 128 filters
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

# Comparison Between ModelConv3D10 and ModelConv3D11

| **Feature**                     | **ModelConv3D10**                                           | **ModelConv3D11**                                           |
|----------------------------------|------------------------------------------------------------|------------------------------------------------------------|
| **Architecture**                | Standard architecture with **one Conv3D layer** per block. | Advanced architecture with **two Conv3D layers** per block. |
| **Number of Parameters**        | Lower parameter count due to single Conv3D layers.         | Higher parameter count due to additional Conv3D layers in each block. |
| **Feature Extraction**          | Less complex features extracted at each block.             | Extracts **more complex and detailed features** with additional layers. |
| **Depth of Network**            | Shallower network with fewer operations per block.         | Deeper network, providing better **hierarchical feature extraction**. |
| **Computational Requirements**  | Computationally lightweight, faster training.              | **Higher computational cost** and longer training times due to extra layers. |
| **Overfitting Risk**            | Lower risk of overfitting due to reduced complexity.        | **Higher risk of overfitting** without sufficient regularization or data. |
| **Generalization**              | Adequate generalization for simpler datasets.              | Better suited for complex datasets with diverse patterns.  |
| **Use of Augmentation**         | Relies on standard augmentation for robustness.            | **Enhanced robustness** when combined with stronger augmentation. |

### Key Notes:
- **ModelConv3D10** is ideal for simpler datasets and resource-constrained environments.
- **ModelConv3D11** is better for complex datasets but requires more computational power and careful regularization.


In [ ]:
conv_3d11=ModelConv3D11()
conv_3d11.initialize_path(project_folder)
conv_3d11.initialize_image_properties(image_height=120,image_width=120)
conv_3d11.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=25)
conv_3d11_model=conv_3d11.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.5)
conv_3d11_model.summary()

In [ ]:
print("Total Params:", conv_3d11_model.count_params())
history_model11=conv_3d11.train_model(conv_3d11_model,augment_data=True)

In [ ]:
plot(history_model11)


In [ ]:
# Model 12 with Augmentation
# Adding dropouts - Similar to Model 5

# Define the ModelConv3D12 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D12(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for filter size, dense neurons, and dropout rate
    def define_model(self, filter_size=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First block: two Conv3D layers with 16 filters
        model.add(Conv3D(16, filter_size, padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(16, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        # Second block: two Conv3D layers with 32 filters
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(32, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        # Third block: two Conv3D layers with 64 filters
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(64, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        # Fourth block: two Conv3D layers with 128 filters
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(128, filter_size, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

**Summary of ModelConv3D12**
- **Purpose:** Model 12 builds on Model 5 by introducing dropout layers after each MaxPooling3D layer to prevent overfitting.
**Key Features:**
 **1.Architecture:**

- Four blocks of **two Conv3D layers** each, followed by MaxPooling3D for downsampling.
- Dropout (rate: 0.25) added after every pooling layer and dense layer for
regularization.
- Fully connected layers (2 Dense layers) to combine features.

  
 **2.Regularization:**

- **Dropout** layers after pooling layers and dense layers help reduce overfitting by randomly deactivating neurons.
  
 **3.Hyperparameters:**

- **Filter size:** (3, 3, 3) across all Conv3D layers.
- **Dense neurons:** 64, with dropout applied in dense layers too.
  
 **4.Compilation:**

- **Optimizer:** Adam with a learning rate of 0.0002.
- **Loss:** Categorical crossentropy for multi-class classification.
- **Metrics:** Categorical accuracy.

 **5.Augmentation Suitability:**

- This model is designed to perform well on augmented data by improving robustness through dropouts.
  
**Use Case:**
- Suitable for datasets prone to **overfitting** or when dealing with smaller datasets.
- Enhances generalization by preventing over-reliance on specific features.


In [ ]:
conv_3d12=ModelConv3D12()
conv_3d12.initialize_path(project_folder)
conv_3d12.initialize_image_properties(image_height=120,image_width=120)
conv_3d12.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=25)
conv_3d12_model=conv_3d12.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.25)
conv_3d12_model.summary()

**Explanation of Code:**
- **1.Model Initialization**
- **2.Model Definition**
**Key Points:**
- **Resolution:** 120x120 image size.
- **Hyperparameters:** 16 frames, batch size 20, 25 epochs.
- **Model:** Defines a 3D convolutional network with large dense layers (256 neurons).
- **Dropout:** Added for regularization (0.25).






In [ ]:
print("Total Params:", conv_3d12_model.count_params())
history_model12=conv_3d12.train_model(conv_3d12_model,augment_data=True)

In [ ]:
plot(history_model12)


In [ ]:
# Comparable to Model 6, Model 13 with Augmentation Reducing network parameters

# Define the ModelConv3D13 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D13(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for dense neurons and dropout rate
    def define_model(self, dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First convolutional block with filter size (3, 3, 3)
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Second convolutional block with filter size (2, 2, 2)
        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Third convolutional block with filter size (2, 2, 2)
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Fourth convolutional block with filter size (2, 2, 2)
        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

**Summary of ModelConv3D13**
- **Purpose:** Model 13 reduces the network parameters compared to Model 6 by using smaller filters in the Conv3D layers.
  
**Key Features:**
 **1.Architecture:**

- **Four convolutional blocks:**
- 1st block: Conv3D with 16 filters (3x3x3), MaxPooling3D (2x2x2).
- 2nd, 3rd, 4th blocks: Conv3D with 32, 64, and 128 filters (2x2x2), MaxPooling3D.
- **BatchNormalization and ReLU activation** after each convolution.
- **Flatten** the 3D output for the fully connected layer.
  
 **2.Fully Connected Layers:**

- Two Dense layers (64 neurons) with dropout (0.25) for regularization.
  
 **3.Regularization:**

- **Dropout** after dense layers to prevent overfitting.
  
 **4.Compilation:**

- **Optimizer:** Adam with learning rate 0.0002.
- **Loss:** Categorical crossentropy.
- **Metrics:** Categorical accuracy.
**Key Differences:
- **Smaller filters (2x2x2):** In contrast to larger filter sizes in previous models, reducing parameters.
- **Efficiency:** Aims to balance performance with fewer parameters, making it suitable for faster training and lower computational cost.
  
**Use Case:**
- Best suited for smaller datasets or limited computational resources.
- Can be used when model simplicity is prioritized without compromising much on performance.

In [ ]:
# Instantiate the ModelConv3D13 class
conv_3d13 = ModelConv3D13()

# Initialize paths and image properties
conv_3d13.initialize_path(project_folder)
conv_3d13.initialize_image_properties(image_height=100, image_width=100)
conv_3d13.initialize_hyperparams(frames_to_sample=16, batch_size=20, num_epochs=25)

# Define the ModelConv3D13 model with specified parameters
conv_3d13_model = conv_3d13.define_model(dense_neurons=128, dropout=0.25)

# Display model summary
conv_3d13_model.summary()

In [ ]:
print("Total Params:", conv_3d13_model.count_params())
history_model13=conv_3d13.train_model(conv_3d13_model,augment_data=True)

In [ ]:
plot(history_model13)


In [ ]:
# Define the ModelConv3D14 class inheriting from ModelBuilderMoreAugmentation
class ModelConv3D14(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for dense neurons and dropout rate
    def define_model(self, dense_neurons=64, dropout=0.25):

        model = Sequential()
        
        # First convolutional block with filter size (3, 3, 3)
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Second convolutional block with filter size (3, 3, 3)
        model.add(Conv3D(32, (3, 3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Third convolutional block with filter size (2, 2, 2)
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        # Fourth convolutional block with filter size (2, 2, 2)
        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        # Flatten the 3D output to 1D
        model.add(Flatten())
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Second fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Output layer with softmax activation for multi-class classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model


### Comparison Between ModelConv3D13 and ModelConv3D14

| **Aspect**             | **ModelConv3D13**                                          | **ModelConv3D14**                                          | **Difference/Impact**                                                                                     |
|------------------------|------------------------------------------------------------|------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Convolutional Layers**| 4 Conv3D blocks with varying filter sizes (16, 32, 64, 128) | 4 Conv3D blocks: 2 with filter size (3x3x3), 2 with (2x2x2) | **ModelConv3D14** uses larger (3x3x3) filters in the first two blocks and smaller (2x2x2) in the last two. |
| **Filter Size**         | (3, 3, 3) for all layers                                   | (3, 3, 3) for first two layers, (2, 2, 2) for last two     | **ModelConv3D14** reduces parameter count in last two layers, which may reduce overfitting and training time. |
| **Dense Layers**        | 2 Dense layers with 64 neurons each                        | 2 Dense layers with 64 neurons each                        | Same architecture, no change.                                                                               |
| **Dropout**             | Dropout after each block and dense layer                   | Dropout after each block and dense layer                   | **No difference** in dropout rate (0.25).                                                                  |
| **Output Layer**        | Softmax for multi-class classification                     | Softmax for multi-class classification                     | **No difference**.                                                                                         |
| **Total Parameters**    | Higher due to more parameters in Conv3D layers             | Slightly fewer due to reduced size in last two Conv3D layers | **ModelConv3D14** may be slightly faster to train due to reduced parameters.                               |
| **MaxPooling3D**        | MaxPooling3D after each Conv3D block                       | MaxPooling3D after each Conv3D block                       | **No difference** in pooling layers.                                                                       |

### Key Differences and Their Impact:
1. **Filter Sizes**: 
   - **ModelConv3D14** uses smaller filters (2x2x2) in the last two convolutional layers compared to **ModelConv3D13**, which uses uniform (3x3x3) filters across all layers. This results in fewer parameters in **ModelConv3D14**, potentially leading to quicker training times and reduced computational cost.
   
2. **Parameter Count**:
   - The reduction in the filter size in **ModelConv3D14** leads to a decrease in the total number of parameters compared to **ModelConv3D13**, which can help with overfitting on smaller datasets.

3. **Model Complexity**:
   - **ModelConv3D13** is more complex, with a uniform filter size across all layers, allowing it to capture more detailed features. However, this can lead to slower training and the risk of overfitting.
   - **ModelConv3D14** is designed to be more efficient by reducing parameters, making it potentially better suited for situations where model training time or computational resources are limited.

### Use Case Considerations:
- **ModelConv3D14** may be more suitable for smaller datasets or when faster training is needed due to its reduced number of parameters.
- **ModelConv3D13**, being more parameter-heavy, could perform better on large, complex datasets where it can take full advantage of the deeper feature extraction capabilities provided by the uniform filter sizes.


In [ ]:


conv_3d14=ModelConv3D14()
conv_3d14.initialize_path(project_folder)
conv_3d14.initialize_image_properties(image_height=120,image_width=120)
conv_3d14.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=25)
conv_3d14_model=conv_3d14

In [ ]:
print("Total Params:", conv_3d14_model.count_params())
history_model14=conv_3d14.train_model(conv_3d14_model,augment_data=True)

In [ ]:
plot(history_model14)


In [ ]:
# Model 15 with Augmentation
# CNN LSTM with GRU - Similar to Model 8

# Define the RNNCNN2 class inheriting from ModelBuilderMoreAugmentation
class RNNCNN2(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for LSTM cells, dense neurons, and dropout rate
    def define_model(self, lstm_cells=64, dense_neurons=64, dropout=0.25):

        model = Sequential()

        # First TimeDistributed convolutional block
        model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'),
                                  input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        # Second TimeDistributed convolutional block
        model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        # Third TimeDistributed convolutional block
        model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        # Fourth TimeDistributed convolutional block
        model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        # Flatten the 2D output to 1D for input to the RNN layer
        model.add(TimeDistributed(Flatten()))

        # GRU layer to capture temporal dependencies
        model.add(GRU(lstm_cells))
        model.add(Dropout(dropout))
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(Dropout(dropout))
        
        # Second fully connected dense layer
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

# RNNCNN2 Model - Combining CNN, GRU, and Dense Layers

The RNNCNN2 model is a hybrid neural network architecture for processing sequential data like videos. It leverages **CNNs** for spatial feature extraction and **GRU** for temporal sequence modeling. Here's a breakdown of the key components:

1. **TimeDistributed CNN Blocks**:
   - Extract spatial features from each frame.
   - Normalization and pooling ensure efficient feature learning.

2. **GRU Layer**:
   - Captures temporal relationships between video frames.

3. **Dense Layers**:
   - Process the temporal features for classification.
   - Includes dropout layers to prevent overfitting.

4. **Key Differences from Model 14**:
   - GRU replaces LSTM, making it computationally efficient.
   - An additional dense layer is added for better feature learning.

5. **Compilation**:
   - Uses the Adam optimizer with a learning rate of 0.0002.
   - Categorical crossentropy as the loss function, suitable for multi-class problems.


In [ ]:
rnn_cnn2=RNNCNN2()
rnn_cnn2.initialize_path(project_folder)
rnn_cnn2.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn2.initialize_hyperparams(frames_to_sample=18,batch_size=20,num_epochs=20)
rnn_cnn2_model=rnn_cnn2.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn2_model.summary()

In [ ]:
print("Total Params:", rnn_cnn2_model.count_params())
history_model15=rnn_cnn2.train_model(rnn_cnn2_model,augment_data=True)

In [ ]:
plot(history_model15)


In [ ]:
# Using the MobileNet model in place of more powerful models like VGG16, Alexnet, InceptionV3, etc. because of its lightweight construction and fast performance. 
# Furthermore, we are currently experiencing low disk space on the nimblebox.ai platform. 

from keras.applications import mobilenet

In [ ]:
# Model 16 

# Import MobileNet and instantiate with pre-trained weights
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

# Define the RNNCNN_TL class inheriting from ModelBuilderMoreAugmentation
class RNNCNN_TL(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for LSTM cells, dense neurons, and dropout rate
    def define_model(self, lstm_cells=64, dense_neurons=64, dropout=0.25):
        
        # Initialize a Sequential model
        model = Sequential()
        
        # Add TimeDistributed MobileNet layer as the convolutional base
        model.add(TimeDistributed(mobilenet_transfer, input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        
        # Freeze MobileNet layers to prevent training
        for layer in model.layers:
            layer.trainable = False
        
        # Batch normalization and max pooling layers
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        # Flatten the output for input to LSTM layer
        model.add(TimeDistributed(Flatten()))

        # LSTM layer to capture temporal dependencies
        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(Dropout(dropout))
        
        # Second fully connected dense layer
        model.add(Dense(self.num_classes, activation='softmax'))
        
        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

# RNNCNN_TL Model - Transfer Learning with MobileNet and LSTM

The RNNCNN_TL model utilizes **transfer learning** to combine the spatial feature extraction power of a pre-trained MobileNet and the temporal modeling capabilities of an LSTM layer. Below are the key components:

1. **MobileNet Transfer Learning**:
   - MobileNet is pre-trained on ImageNet and used as a feature extractor.
   - The top classification layers of MobileNet are removed, and the weights are frozen to retain its pre-trained features.

2. **TimeDistributed Wrapper**:
   - Ensures that MobileNet processes each video frame independently while preserving the sequence.

3. **BatchNormalization and MaxPooling**:
   - Normalize the extracted features and reduce spatial dimensions.

4. **Flattening**:
   - Converts 2D feature maps into 1D vectors for the LSTM layer.

5. **LSTM Layer**:
   - Captures temporal dependencies across the sequence of frames.

6. **Fully Connected Dense Layers**:
   - First dense layer refines features with ReLU activation.
   - Second dense layer outputs probabilities for each class using softmax activation.

7. **Key Characteristics**:
   - Leverages transfer learning for efficient spatial feature extraction.
   - Regularization with dropout to prevent overfitting.
   - Uses Adam optimizer and categorical crossentropy loss for training.

This architecture is well-suited for video classification tasks, combining the strengths of pre-trained CNNs and RNNs.


In [ ]:
rnn_cnn_tl=RNNCNN_TL()
rnn_cnn_tl.initialize_path(project_folder)
rnn_cnn_tl.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn_tl.initialize_hyperparams(frames_to_sample=16,batch_size=5,num_epochs=20)
rnn_cnn_tl_model=rnn_cnn_tl.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn_tl_model.summary()

In [ ]:
print("Total Params:", rnn_cnn_tl_model.count_params())
history_model16=rnn_cnn_tl.train_model(rnn_cnn_tl_model,augment_data=True)

In [ ]:
plot(history_model16)


In [ ]:
# Import MobileNet from Keras applications
from keras.applications import mobilenet

# Load MobileNet pre-trained on ImageNet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

# Define the RNNCNN_TL2 class inheriting from ModelBuilderMoreAugmentation
class RNNCNN_TL2(ModelBuilderMoreAugmentation):
    
    # Define the model with parameters for GRU cells, dense neurons, and dropout rate
    def define_model(self, gru_cells=64, dense_neurons=64, dropout=0.25):
        
        # Initialize a Sequential model
        model = Sequential()
        
        # Add TimeDistributed MobileNet layer as the convolutional base
        model.add(TimeDistributed(mobilenet_transfer, input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
 
        # Batch normalization and max pooling layers
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        model.add(TimeDistributed(Flatten()))

        # GRU layer to capture temporal dependencies
        model.add(GRU(gru_cells))
        model.add(Dropout(dropout))
        
        # First fully connected dense layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(Dropout(dropout))
        
        # Second fully connected dense layer
        model.add(Dense(self.num_classes, activation='softmax'))
        
        # Compile the model with Adam optimizer and categorical crossentropy loss
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        
        return model

# RNNCNN_TL2 Model - Transfer Learning with MobileNet and GRU

The **RNNCNN_TL2** model is an enhanced architecture that combines **MobileNet** as a pre-trained convolutional base and a **GRU** (Gated Recurrent Unit) for modeling temporal dependencies in sequential data. This model is particularly effective for tasks like video classification. Here's a breakdown of its components:

---

### **Key Components of RNNCNN_TL2**

1. **MobileNet as Convolutional Base**:
   - **Purpose**: Leverage the spatial feature extraction power of MobileNet pre-trained on the ImageNet dataset.
   - **Details**:
     - The MobileNet model is imported from Keras and its top classification layers are excluded (`include_top=False`).
     - MobileNet processes individual frames of a video independently using the `TimeDistributed` wrapper.

2. **TimeDistributed Wrapper**:
   - **Purpose**: Ensure that MobileNet and subsequent layers process each frame in a sequence independently.
   - **Details**:
     - Wraps MobileNet and subsequent layers to maintain the temporal sequence structure.

3. **BatchNormalization and MaxPooling**:
   - **Purpose**:
     - `BatchNormalization`: Normalizes feature activations, improving convergence and stabilizing training.
     - `MaxPooling2D`: Reduces spatial dimensions to focus on the most salient features.

4. **Flatten Layer**:
   - **Purpose**: Converts the 2D feature maps from MobileNet into 1D vectors for input to the GRU layer.

5. **GRU Layer**:
   - **Purpose**: Captures temporal dependencies and patterns across the sequence of frames.
   - **Details**:
     - GRUs are efficient and perform well for sequences with temporal relationships.
     - The number of GRU units (`gru_cells`) is configurable (default: 64).

6. **Dropout Layers**:
   - **Purpose**: Prevent overfitting by randomly dropping connections during training.

7. **Fully Connected Dense Layers**:
   - **Purpose**:
     - The first dense layer learns high-level representations using ReLU activation.
     - The second dense layer outputs class probabilities using `softmax` activation.
   - **Details**:
     - Configurable number of neurons in the first dense layer (`dense_neurons`, default: 64).

8. **Compilation**:
   - **Optimizer**: Adam optimizer for adaptive learning rate optimization.
   - **Loss Function**: Categorical Crossentropy, ideal for multi-class classification problems.
   - **Metric**: Categorical accuracy to evaluate performance.

---

### **Summary**
- **MobileNet** provides powerful spatial feature extraction without the need for extensive training.
- **GRU** models temporal relationships efficiently with fewer parameters compared to LSTMs.
- Regularization through **dropout layers** ensures robustness and prevents overfitting.
- The model is well-suited for sequential tasks like video classification, combining the strengths of transfer learning and RNNs.



In [ ]:
# Initialize an instance of the RNNCNN_TL2 class
rnn_cnn_tl2 = RNNCNN_TL2()

# Initialize paths, image properties, and hyperparameters
rnn_cnn_tl2.initialize_path(project_folder)
rnn_cnn_tl2.initialize_image_properties(image_height=120, image_width=120)
rnn_cnn_tl2.initialize_hyperparams(frames_to_sample=16, batch_size=5, num_epochs=20)

# Define the RNNCNN_TL2 model with specified parameters
rnn_cnn_tl2_model = rnn_cnn_tl2.define_model(gru_cells=128, dense_neurons=128, dropout=0.25)

# Print the summary of the RNNCNN_TL2 model
rnn_cnn_tl2_model.summary()

In [ ]:
print("Total Params:", rnn_cnn_tl2_model.count_params())
history_model17=rnn_cnn_tl2.train_model(rnn_cnn_tl2_model,augment_data=True)

In [ ]:
plot(history_model17)


In [ ]:
import time
from keras.models import load_model

# Load the saved model
# The `load_model` function from Keras is used to load a pre-trained model from a specified file path.
# In this case, the model file is named 'model-00001-1.34029-0.43137-1.75732-0.28000.h5' and is located
# within the directory 'model_init_2024-05-2816_20_05.434836'. The numbers in the file name might represent
# different metrics such as loss and accuracy during training, typically indicating epoch number, training
# loss, training accuracy, validation loss, and validation accuracy respectively.
model = load_model('model_init_2024-06-0407_48_54.745934/model-00001-1.34029-0.43137-1.75732-0.28000.h5')

In [ ]:
# Initialize the test generator
# `RNNCNN1` is a class that is presumably defined elsewhere in your code. It is used to create an instance of a data generator
# for testing purposes. This generator likely prepares and feeds data in a specific format suitable for RNN and CNN models.
test_generator = RNNCNN1()

# Initialize the path for the test generator
# The `initialize_path` method is called with `project_folder` to set up the directory or path where the data is located.
test_generator.initialize_path(project_folder)

# Set image properties
# The `initialize_image_properties` method sets the height and width of the images that the generator will process.
# Here, the images are set to be 120x120 pixels.
test_generator.initialize_image_properties(image_height=120, image_width=120)

# Set hyperparameters
# The `initialize_hyperparams` method sets various hyperparameters for the generator.
# - `frames_to_sample`: Number of frames to sample from the video or sequence.
# - `batch_size`: Number of samples per batch of computation.
# - `num_epochs`: Number of epochs for which the generator will run.
test_generator.initialize_hyperparams(frames_to_sample=18, batch_size=20, num_epochs=20)

# Create a data generator
# The `generator` method of `test_generator` is called to create an actual data generator.
# - `test_generator.val_path` and `test_generator.val_doc` presumably specify the validation data's path and documents respectively.
# - `augment=False` indicates that data augmentation (like random transformations) is not applied.
g = test_generator.generator(test_generator.val_path, test_generator.val_doc, augment=False)

# Fetch the next batch of data
# The `next` function is used to get the next batch of data from the generator `g`.
# `batch_data` contains the data samples in the batch, and `batch_labels` contains the corresponding labels.
batch_data, batch_labels = next(g)